## STEP 1 — Helper: Deteksi Layer & Namespace

In [1]:
import re
from pathlib import Path

def detect_layer(file_path: str) -> str:
    path = file_path.lower()

    if "domain" in path:
        return "Domain"
    if "application" in path:
        return "Application"
    if "infrastructure" in path:
        return "Infrastructure"
    if "api" in path or "presentation" in path:
        return "Presentation"

    return "Unknown"



def extract_namespace(content: str) -> str:
    match = re.search(r"namespace\s+([\w\.]+)", content)
    return match.group(1) if match else "Unknown"

## 📥 STEP 2 — Loader C# (Clean Architecture Aware)

In [2]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    "../src",
    glob="**/*.cs",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)

docs_csharp = loader.load()
docs_csharp


[Document(metadata={'source': '..\\src\\Application\\Program.cs'}, page_content='\ufeff// See https://aka.ms/new-console-template for more information\nConsole.WriteLine("Hello, World!");\n'),
 Document(metadata={'source': '..\\src\\Domain\\Program.cs'}, page_content='\ufeff// See https://aka.ms/new-console-template for more information\nConsole.WriteLine("Hello, World!");\n'),
 Document(metadata={'source': '..\\src\\Infrastructure\\Program.cs'}, page_content='\ufeff// See https://aka.ms/new-console-template for more information\nConsole.WriteLine("Hello, World!");\n'),
 Document(metadata={'source': '..\\src\\Api\\OrderService\\Program.cs'}, page_content='using Microsoft.EntityFrameworkCore;\nusing static System.Net.Mime.MediaTypeNames;\n\nvar builder = WebApplication.CreateBuilder(args);\n\n// Configuration: load connection string\nvar conn = builder.Configuration.GetConnectionString("DefaultConnection") ??\n           "Host=postgres;Database=postgres;Username=postgres;Password=P@ssw0

## 🏷️ STEP 3 — Enrich Metadata (Layer + Namespace)

In [3]:
from langchain_core.documents import Document

documents: list[Document] = []

for doc in docs_csharp:
    content = doc.page_content
    path = doc.metadata.get("source", "")

    doc.metadata.update({
        "layer": detect_layer(path),
        "namespace": extract_namespace(content),
        "file_path": path
    })

    documents.append(doc)


documents

[Document(metadata={'source': '..\\src\\Application\\Program.cs', 'layer': 'Application', 'namespace': 'Unknown', 'file_path': '..\\src\\Application\\Program.cs'}, page_content='\ufeff// See https://aka.ms/new-console-template for more information\nConsole.WriteLine("Hello, World!");\n'),
 Document(metadata={'source': '..\\src\\Domain\\Program.cs', 'layer': 'Domain', 'namespace': 'Unknown', 'file_path': '..\\src\\Domain\\Program.cs'}, page_content='\ufeff// See https://aka.ms/new-console-template for more information\nConsole.WriteLine("Hello, World!");\n'),
 Document(metadata={'source': '..\\src\\Infrastructure\\Program.cs', 'layer': 'Infrastructure', 'namespace': 'Unknown', 'file_path': '..\\src\\Infrastructure\\Program.cs'}, page_content='\ufeff// See https://aka.ms/new-console-template for more information\nConsole.WriteLine("Hello, World!");\n'),
 Document(metadata={'source': '..\\src\\Api\\OrderService\\Program.cs', 'layer': 'Presentation', 'namespace': 'Unknown', 'file_path': '.

## 🧩 STEP 4 — Chunking (Aman untuk Code)

In [4]:
import re

from langchain_text_splitters import RecursiveCharacterTextSplitter

## STEP 1 — Regex Detector C# Symbol
TYPE_REGEX = re.compile(
    r"""
    (public|internal|private|protected)?\s*
    (abstract|sealed|static)?\s*
    (class|interface|record)\s+
    (?P<name>\w+)
    """,
    re.VERBOSE
)

METHOD_REGEX = re.compile(
    r"""
    (public|internal|private|protected)?\s*
    (async|static|virtual|override)?\s*
    [\w\<\>\[\]]+\s+
    (?P<name>\w+)\s*
    \(
    """,
    re.VERBOSE
)
##STEP 2 — Ambil Block {} dengan Brace Counter
def extract_block(text: str, start_index: int) -> tuple[str, int]:
    brace_count = 0
    i = start_index

    while i < len(text):
        if text[i] == "{":
            brace_count += 1
        elif text[i] == "}":
            brace_count -= 1

        if brace_count == 0 and i > start_index:
            return text[start_index:i+1], i+1

        i += 1

    return text[start_index:], len(text)

##STEP 3 — Auto Chunk per Class / Interface
from langchain_core.documents import Document

def chunk_by_type(doc: Document, split_methods=False) -> list[Document]:
    content = doc.page_content
    chunks = []

    for match in TYPE_REGEX.finditer(content):
        start = content.find("{", match.end())
        if start == -1:
            continue

        block, _ = extract_block(content, start)

        symbol_name = match.group("name")
        symbol_type = match.group(3)

        base_metadata = {
            **doc.metadata,
            "symbol_type": symbol_type,
            "symbol_name": symbol_name
        }

        # === METHOD SPLITTING (OPTIONAL) ===
        if split_methods and symbol_type == "class":
            method_chunks = chunk_methods(block, base_metadata)
            chunks.extend(method_chunks)
        else:
            chunks.append(
                Document(
                    page_content=block,
                    metadata=base_metadata
                )
            )

    return chunks

def chunk_methods(class_block: str, base_metadata: dict) -> list[Document]:
    results = []

    for match in METHOD_REGEX.finditer(class_block):
        start = class_block.find("{", match.end())
        if start == -1:
            continue

        block, _ = extract_block(class_block, start)

        results.append(
            Document(
                page_content=block,
                metadata={
                    **base_metadata,
                    "symbol_type": "method",
                    "method_name": match.group("name")
                }
            )
        )

    return results

## STEP 5 — Apply ke Seluruh Documents
auto_chunks: list[Document] = []

for doc in documents:
    auto_chunks.extend(
        chunk_by_type(
            doc,
            split_methods=True  # 🔥 TRUE = method-level RAG
        )
    )


print(f"Total chunks: {len(auto_chunks)}")
print(auto_chunks)


Total chunks: 21
[Document(metadata={'source': '..\\src\\Api\\OrderService\\Controllers\\OrdersController.cs', 'layer': 'Presentation', 'namespace': 'Unknown', 'file_path': '..\\src\\Api\\OrderService\\Controllers\\OrdersController.cs', 'symbol_type': 'method', 'symbol_name': 'OrdersController', 'method_name': 'OrdersController'}, page_content='{ _mediator = mediator; }'), Document(metadata={'source': '..\\src\\Api\\OrderService\\Controllers\\OrdersController.cs', 'layer': 'Presentation', 'namespace': 'Unknown', 'file_path': '..\\src\\Api\\OrderService\\Controllers\\OrdersController.cs', 'symbol_type': 'method', 'symbol_name': 'OrdersController', 'method_name': 'Create'}, page_content='{\n        var id = await _mediator.Send(cmd);\n        return CreatedAtAction(nameof(Get), new { id }, new { id });\n    }'), Document(metadata={'source': '..\\src\\Api\\OrderService\\Controllers\\OrdersController.cs', 'layer': 'Presentation', 'namespace': 'Unknown', 'file_path': '..\\src\\Api\\OrderSer

In [5]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

vectorstore = Chroma.from_documents(
    documents=auto_chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    persist_directory="../chroma_clean_arch"
)
vectorstore.persist()

C:\Users\wibow\AppData\Local\Temp\ipykernel_13192\4100783405.py:6: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="nomic-embed-text"),
C:\Users\wibow\AppData\Local\Temp\ipykernel_13192\4100783405.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


## STEP 6 — Retriever DENGAN FILTER LAYER
Contoh: Application layer + method only

In [6]:
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 5,
        "filter": {
            "layer": "Application",
            "symbol_type": "method"
        }
    }
)

print(retriever)




tags=['Chroma', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002E878272C90> search_kwargs={'k': 5, 'filter': {'layer': 'Application', 'symbol_type': 'method'}}


In [7]:
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 5,
        "filter": {
            "layer": "Domain",
            "symbol_type": "interface"
        }
    }
)
print(retriever)

tags=['Chroma', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002E878272C90> search_kwargs={'k': 5, 'filter': {'layer': 'Domain', 'symbol_type': 'interface'}}


In [8]:
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 6,
        "filter": {"layer": {"$in": ["Domain", "Application"]}}
    }
)

print(retriever)

tags=['Chroma', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002E878272C90> search_kwargs={'k': 6, 'filter': {'layer': {'$in': ['Domain', 'Application']}}}


## 🤖 STEP 7 — RAG Chain (Ollama Chat)

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama


llm = ChatOllama(
    model="llama3.1",
    temperature=0
)

prompt = ChatPromptTemplate.from_template("""
Kamu adalah senior C# architect.
Ikuti Clean Architecture:
- Domain tidak bergantung ke layer lain
- Application berisi Use Case
- Infrastructure hanya implementasi

1. Gunakan pola yang SUDAH ADA di repository.
2. Gunakan struktur AST berikut untuk membuat Use Case

Context:
{context}

Question:
{question}
""")

retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 6,
        "filter": {
            "layer": {"$in": ["Domain", "Application"]}
        }
    }
)

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)



C:\Users\wibow\AppData\Local\Temp\ipykernel_13192\487001055.py:6: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [10]:
query_orderApproval = """
Buatkan Use Case untuk Approve Order
berdasarkan pola yang sudah ada di Application layer
dan entity di Domain
"""
result = rag_chain.invoke(
   query_orderApproval
)

print(result.content)

Berikut adalah contoh implementasi Use Case untuk Approve Order berdasarkan pola yang sudah ada di Application layer dan entity di Domain:

**Domain**
```csharp
// IOrderRepository.cs
public interface IOrderRepository
{
    Task<Order?> GetByIdAsync(Guid id);
    Task AddAsync(Order order);
    Task SaveChangesAsync(CancellationToken ct = default);
}

// Order.cs
public class Order
{
    public Guid Id { get; set; }
    public string Status { get; set; }

    public void Approve()
    {
        Status = "Approved";
    }
}
```

**Application**
```csharp
// IOrderService.cs (pola yang sudah ada)
public interface IOrderService
{
    Task<Order?> GetByIdAsync(Guid id);
    Task AddAsync(Order order);
    Task SaveChangesAsync(CancellationToken ct = default);
}

// OrderService.cs
public class OrderService : IOrderService
{
    private readonly IOrderRepository _repository;

    public OrderService(IOrderRepository repository)
    {
        _repository = repository;
    }

    public async

In [ ]:
result = rag_chain.invoke(
    "Buatkan Use Case CreateOrder sesuai pola project ini"
)

print(result.content)

In [16]:
result = rag_chain.invoke(
    "apakah terdapat Order Item ?")

print(result.content)

Dengan menggunakan Clean Architecture dan Repository pattern, kita dapat memisahkan domain logic dari implementasi infrastruktur.

Pertama-tama, kita perlu membuat interface untuk repository yang akan digunakan oleh domain. Misalnya, kita buat interface `IOrderRepository` seperti berikut:

```csharp
public interface IOrderRepository
{
    Task<Order?> GetByIdAsync(Guid id);
    Task AddAsync(Order order);
    Task SaveChangesAsync(CancellationToken ct = default);
}
```

Kemudian, kita perlu membuat implementasi dari repository tersebut. Misalnya, kita buat class `OrderRepository` yang akan mengimplementasikan interface `IOrderRepository`. Kita dapat menggunakan pola Repository pattern seperti berikut:

```csharp
public class OrderRepository : IOrderRepository
{
    private readonly DbContext _context;

    public OrderRepository(DbContext context)
    {
        _context = context;
    }

    public async Task<Order?> GetByIdAsync(Guid id)
    {
        return await _context.Orders.Find

In [11]:
query_orderApproval = """
Business Requirement Design for BOOK Library Application

Introduction
This business requirement document outlines the key functionalities and design principles that will form the foundation of the BOOK Library application. The application aims to provide an intuitive and user-friendly platform for users to access and manage a wide range of book resources.

Objectives
To create an accessible digital library with a comprehensive collection of books.
To provide easy navigation and efficient book management for both users and administrators.
To integrate advanced search capabilities for quick retrieval of books and information.
To enable social features that allow user interactions and community engagement.

Functional Requirements

User Accounts
User Registration: Allow new users to create an account using their email address.
Profile Management: Enable users to edit their profile details and preferences.

Catalog Management
Book Inventory: Maintain a detailed inventory of all books available in the library.
Categories and Tags: Organize books into various categories and tags for easy browsing.
Book Details: Each book entry should contain details such as title, author, ISBN, and a brief synopsis.

Search Functionality
Basic Search: Users can search for books by title, author, or ISBN.
Advanced Search: Users can perform complex searches using filters like genre, publication date, and user ratings.

Borrowing System
Checkout Process: Implement a system for checking out books for a specified loan period.
Renewals and Returns: Allow users to renew loans or return books through the application interface.

Reservation System
Book Reservations: Users can reserve books that are currently on loan.
Notification System: Notify users when a reserved book becomes available.

User Interaction
Ratings and Reviews: Users can rate books and leave reviews for others to read.
Recommendation Engine: Suggest books to users based on their reading history and preferences.

Administrative Features
Dashboard: Provide admins with a dashboard to monitor library metrics and user activities.
User Management: Enable administrators to manage user accounts and privileges.
Book Management: Allow administrators to add, update, or remove books from the catalog.

"""
result = rag_chain.invoke(
   query_orderApproval
)

print(result.content)

Berikut adalah desain arsitektur Clean Architecture untuk aplikasi BOOK Library:

**Domain Layer**

* **Entity**: Book, User, Reservation, Loan
	+ Book memiliki atribut seperti title, author, ISBN, dan synopsis
	+ User memiliki atribut seperti email, nama, alamat, dll.
	+ Reservation memiliki atribut seperti tanggal reservasi, status reservasi (dipenuhi atau tidak)
	+ Loan memiliki atribut seperti tanggal pinjam, tanggal kembali, status loan (dipinjam atau tidak dipinjam)
* **Value Object**: BookTitle, AuthorName, ISBN
	+ Value object digunakan untuk menyimpan nilai-nilai yang tidak dapat berubah

**Application Layer**

* **Use Case**: 
	+ UserRegistration: memungkinkan pengguna mendaftar dengan email
	+ ProfileManagement: memungkinkan pengguna mengedit profil dan preferensi
	+ CatalogManagement:
		- BookInventory: memelihara inventori buku yang tersedia di perpustakaan
		- CategoriesAndTags: mengorganisir buku ke dalam kategori dan tag untuk pencarian mudah
		- BookDetails: setiap ent

In [12]:
query_orderApproval = """
buatkan code dengan desain arsitektur Clean Architecture untuk aplikasi BOOK Library:

**Domain Layer**

* **Entity**: Book, User, Reservation, Loan
	+ Book memiliki atribut seperti title, author, ISBN, dan synopsis
	+ User memiliki atribut seperti email, nama, alamat, dll.
	+ Reservation memiliki atribut seperti tanggal reservasi, status reservasi (dipenuhi atau tidak)
	+ Loan memiliki atribut seperti tanggal pinjam, tanggal kembali, status loan (dipinjam atau tidak dipinjam)
* **Value Object**: BookTitle, AuthorName, ISBN
	+ Value object digunakan untuk menyimpan nilai-nilai yang tidak dapat berubah

**Application Layer**

* **Use Case**: 
	+ UserRegistration: memungkinkan pengguna mendaftar dengan email
	+ ProfileManagement: memungkinkan pengguna mengedit profil dan preferensi
	+ CatalogManagement:
		- BookInventory: memelihara inventori buku yang tersedia di perpustakaan
		- CategoriesAndTags: mengorganisir buku ke dalam kategori dan tag untuk pencarian mudah
		- BookDetails: setiap entri buku harus berisi detail seperti judul, penulis, ISBN, dan sinopsis singkat
	+ SearchFunctionality:
		- BasicSearch: pengguna dapat mencari buku dengan judul, penulis, atau ISBN
		- AdvancedSearch: pengguna dapat melakukan pencarian kompleks menggunakan filter seperti genre, tanggal publikasi, dan rating pengguna
	+ BorrowingSystem:
		- CheckoutProcess: implementasikan sistem untuk meminjam buku selama periode pinjaman tertentu
		- RenewalsAndReturns: memungkinkan pengguna untuk memperbarui atau mengembalikan buku melalui antarmuka aplikasi
	+ ReservationSystem:
		- BookReservations: pengguna dapat memesan buku yang sedang dipinjam
		- NotificationSystem: memberitahu pengguna ketika buku yang dipesan tersedia

**Infrastructure Layer**

* **Repository**: implementasikan repository untuk mengakses data di database
* **Database**: menggunakan database untuk menyimpan data aplikasi

**Pola yang SUDAH ADA di Repository**

* Menggunakan pola Repository Pattern untuk mengakses data di database
* Menggunakan Entity Framework Core sebagai ORM (Object-Relational Mapping) untuk menghubungkan domain layer dengan infrastructure layer

**Struktur AST**

* Menggunakan struktur AST (Abstract Syntax Tree) untuk mewakili kode aplikasi dalam bentuk yang dapat dipahami oleh mesin
* Membuat Use Case menggunakan pola Command Pattern untuk memisahkan logika bisnis dari implementasi teknis


"""
result = rag_chain.invoke(
   query_orderApproval
)

print(result.content)

Berikut adalah contoh kode yang sesuai dengan desain arsitektur Clean Architecture yang Anda berikan:

**Domain Layer**

```csharp
// Entity
public class Book
{
    public string Title { get; set; }
    public string Author { get; set; }
    public string ISBN { get; set; }
    public string Synopsis { get; set; }
}

public class User
{
    public string Email { get; set; }
    public string Name { get; set; }
    public string Address { get; set; }
}

public class Reservation
{
    public DateTime DateReservation { get; set; }
    public bool IsReserved { get; set; }
}

public class Loan
{
    public DateTime DateBorrowed { get; set; }
    public DateTime DateReturned { get; set; }
    public bool IsBorrowed { get; set; }
}

// Value Object
public class BookTitle : ValueObject<Book>
{
    public string Title { get; private set; }

    public BookTitle(string title)
    {
        Title = title;
    }
}

public class AuthorName : ValueObject<Author>
{
    public string Name { get; priva